# 要約 
このノートブックは、KaggleのLMSYS - Chatbot Arenaコンペティションにおいて、大規模言語モデル（LLM）の応答の好ましさを予測するタスクに取り組んでいます。特に、LLMの応答がどちらのモデル（モデルAまたはモデルB）がより好まれるかを判定するための推論を行っています。

### 主要な取り組みと手法

1. **高速化された推論**:
   - 元のスクリプトに対して38%の推論時間短縮を達成しました。具体的には、トレーニングセットの最初の10,000サンプルにおける推論時間を65分から40分に短縮しました。
   - これを実現するために、二つの主要な技術を導入しました：
     - **動的パディング**: 各ミニバッチの最長シーケンスに合わせてオンザフライでパディングを行う手法。
     - **テストデータ長によるソート**: 入力の長さでソートすることで各ミニバッチ内のトークン数を揃え、余分なパディングを避けています。

2. **長い入力シーケンスの扱い**: 
   - `max_length`を1024から1280に変更することで、モデルのパフォーマンスが向上しました。

3. **使用ライブラリ**:
   - **PyTorch**、**Transformers**、**Tokenizers**、および**PEFT**（Parameter-Efficient Fine-Tuning）などの機械学習用のライブラリを使用しています。
   - 特に、Transformersライブラリを使ってLLMのトークナイゼーションやモデルの初期化を行っています。

4. **推論プロセスの実装**:
   - 複数のGPUを使用し、並列処理により推論を行うことで効率を向上させています。
   - モデルからの出力を用いて、各モデルの勝利の確率を計算し、最終的な結果を出力します。

5. **提出用ファイルの作成**:
   - 出力された確率をもとに「submission.csv」という形式でCSVファイルを生成し、Kaggleに提出可能なフォーマットに適合させています。

このノートブックは、効率的な推論手法を取り入れ、特にモデルのパフォーマンスを維持しつつ迅速に結果を生成することに焦点を当てています。

---


# 用語概説 
以下は、ノートブック特有のドメイン知識や実務で馴染みの少ない専門用語の簡単な解説です。

### 動的パディング
- **解説**: トレーニングデータ中の各ミニバッチにおいて、最長のシーケンスに合わせてその都度パディングを行う技術。これにより、固定長の入力に対する無駄な計算を減らし、推論時間を短縮できる。

### テスト時間の拡張 (TTA)
- **解説**: テスト時間の拡張（Test Time Augmentation）は、同じサンプルに対して複数の推論を行い、その結果を組み合わせて最終的な予測を行う手法。具体的には、応答の順序を変えたモデル呼び出しのことで、モデルの堅牢性を向上させるために行う。

### ロジット
- **解説**: モデルの出力層から得られる生のスコアで、通常は分類タスクにおいて各クラスに対する非正規化の確率を表す。スコアはソフトマックス関数を通して確率に変換される。

### アテンションマスク
- **解説**: トークンがどの部分で重要であるかを示す二値配列。通常、パディングされた部分に対して0、重要なトークンに対して1を設定して、モデルが無視すべき部分を明示する。

### PEFT (Parameter-Efficient Fine-Tuning)
- **解説**: より少ないパラメータでモデルを微調整する手法。これは、フルモデルを再訓練することなく任意のタスクに適応させたり、効率的にリソースを使用したりするために利用される。

### LoRA (Low-Rank Adaptation)
- **解説**: モデルの低ランク表示を利用して、パラメータを効率的に調整する技術。これにより、少ないデータでモデルのパフォーマンスを向上させることが可能になる。

### スパースドット製品 (SDP)
- **解説**: スパース（まばら）な行列に対して効率的にドット積（内積）を計算するための手法。特に、メモリ使用量と計算コストを削減するために、特定の条件下での計算を最適化する。

この解説が、初心者がつまずく可能性のある用語を理解する助けになれば幸いです。

---


## 🦙🦙🦙 このノートブックについて
このノートブックは、@kishanvavdaraによる [Inference - llama-3 8b](https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b) を元に作成されています。リンクされたノートブックをまだ確認していない方は、ぜひチェックして高評価をつけることをお勧めします。
私は、@kishanvavdaraの作品に対していくつかの改善を加えました：

### 38%高速化された推論
トレーニングセットの最初の10,000サンプルを使用した推論時間は、このスクリプトを使うと40分かかります（TTAなし）。一方、元のスクリプトでは65分かかるため、精度に劣化がないまま38%高速化されました。私が主に追加したのは2つの機能です：

#### 1. 動的パディング
すべての入力を事前に固定長にパディングするのではなく、各ミニバッチの最長のシーケンスに合わせてオンザフライでパディングを適用します。

#### 2. テストデータを入力の長さでソート
動的パディングの利点を最大限に活用するために、テストデータは入力の長さでソートされます。こうすることで、各ミニバッチ内の入力がほぼ同じ長さになり、不要なパディングが減ります。

### より長い入力シーケンス
トレーニングデータの99%は1024トークン以内に収まっていますが、残りの1%はそれを超えています。また、テストセットにはさらに長いシーケンスが存在する可能性があるため、`max_length`をできるだけ長く設定する方が安全だと考えました。
`max_length`を1024から1280に変更すると、LBは0.989から0.983に改善されました。

## 試したが効果がなかったこと

### テスト時間の拡張 (TTA)
私は、response_aとresponse_bの順序を入れ替える簡単なTTAを試みました。この方法では、サンプルごとにモデルが2回呼び出されるため、推論時間が2倍に増加することに注意してください。
2つのソフトマックス確率を平均化するか、2つのロジットを平均してからソフトマックス確率を計算することができます。どちらのアプローチもLBを改善しませんでしたが、ソフトマックスを平均化する方がパフォーマンスが良かったです。
TTAは推論時間を2倍に増加させるため、サンプルごとにモデルを2回呼び出します。効率的な推論のおかげで、`max_length=1280`でTTAを有効にした状態でも、9時間以内に提出が完了しました。

### 各入力の切り捨て
元の実装では、プロンプト + response_a + response_bとして連結されたシーケンスを切り捨てています。切り捨てを単純に適用すると、一部の（稀ではありますが）プロンプトが1280トークンを超えるため、プロンプトのみの入力が生成され、モデルは勝者をランダムに推測するしかなくなります。
私は各入力を固定長にまず切り捨て、その後3つを連結する方法を試しましたが、LBは改善されませんでした。

# ライブラリのインポート

In [ ]:
# bitsandbytesライブラリを最新バージョンにインストールします。 
# -qオプションは出力を抑制し、--no-indexオプションはPyPIインデックスを無視してローカルのリンクを使用します。
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/

# transformersライブラリを最新バージョンにインストールします。
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/

# tokenizersライブラリを最新バージョンにインストールします。
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/

# peftライブラリを最新バージョンにインストールします。
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

In [ ]:
# 必要なライブラリをインポートします

import time  # 時間計測のためのライブラリ
from dataclasses import dataclass  # データクラスを使用するためのライブラリ
from concurrent.futures import ThreadPoolExecutor  # スレッドプールを使用して並行処理を行うためのライブラリ

import torch  # PyTorchライブラリ
import sklearn  # 機械学習ライブラリ
import numpy as np  # 数値計算ライブラリ
import pandas as pd  # データ操作ライブラリ
from transformers import AutoTokenizer, LlamaForSequenceClassification, BitsAndBytesConfig  # Hugging FaceのTransformersライブラリから必要なクラスをインポート
from transformers.data.data_collator import pad_without_fast_tokenizer_warning  # パディングを行う関数をインポート
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType  # PEFT関連のクラスや関数をインポート

In [ ]:
# 使用可能なGPUの数が2つであることを確認します。
# 要求された条件を満たさない場合、エラーメッセージが表示されます。
assert torch.cuda.device_count() == 2, "申し訳ありませんが、マルチGPUが必要です！"

# CUDAのメモリ効率的なスパースドット製品（SDP）を無効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)

# CUDAのフラッシュスパースドット製品（SDP）を無効にします。
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
@dataclass
class Config:
    # 使用するモデルの名前（またはファイルパス）を指定します。
    model_name = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
    
    # モデルの重みの保存場所を指定します。
    weights_path = '/kaggle/input/lmsys-model/model'
    
    # 最大入力シーケンスの長さを指定します。
    max_length = 1280
    
    # バッチサイズを指定します。1回の処理で扱うサンプルの数です。
    batch_size = 8
    
    # 使用するデバイスを指定します。ここではCUDA（GPU）を指定しています。
    device = torch.device("cuda")    
    
    # テスト時間の拡張（TTA）を使用するかどうかを指定します。
    # <prompt>-<model-bの応答>-<model-aの応答>の形式で使用されます。
    tta = False  
    
    # 各入力にmax_length//3を適用するか、連結した入力にmax_lengthを適用するかを指定します。
    spread_max_length = False  

# Configクラスのインスタンスを作成します。
cfg = Config()

# データの準備

In [ ]:
# テストデータをCSVファイルから読み込みます。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# 文字列のリストを連結する関数を定義します。
def process(input_str):
    # 引数として渡された文字列の両端のブラケットを取り除きます。
    stripped_str = input_str.strip('[]')
    # 各文を取り出し、ダブルクオーテーションを取り除いてリストを作成します。
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    # リスト内の文を空白で結合して1つの文字列にします。
    return ' '.join(sentences)

# 'prompt'カラム、'response_a'カラム、'response_b'カラムに対してprocess関数を適用します。
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

# 処理したデータを表示します。最初の5行を表示します。
display(test.head(5))

# トークナイズ（単語分割）

In [ ]:
# トークナイザーを使用してテキストをトークンに変換する関数を定義します。
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    # ユーザーのプロンプトに接頭辞を追加します。
    prompt = ["User prompt: " + p for p in prompt]
    # モデルAの応答に接頭辞を追加します。
    response_a = ["\n\nModel A :\n" + r_a for r_a in response_a]
    # モデルBの応答に接頭辞を追加します。
    response_b = ["\n\n--------\n\nModel B:\n" + r_b for r_b in response_b]
    
    if spread_max_length:
        # max_lengthを3で割った値を使用して個々のトークンの長さを設定します。
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        
        # プロンプトと応答のトークンを結合してinput_idsを作成します。
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        # attention_maskを作成します。各トークンの有効性を示します。
        attention_mask = [[1] * len(i) for i in input_ids]
    else:
        # プロンプトと応答を結合して1つのテキストリストを作成します。
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        # 一括でトークナイザーを使ってトークン化します。
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        # input_idsとattention_maskを取得します。
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    
    return input_ids, attention_mask  # トークンIDとアテンションマスクを返します。

In [ ]:
%%time  # このセルの実行時間を計測します。

# トークナイザーを事前学習済みモデルからロードします。
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# データフレームを初期化します。
data = pd.DataFrame()
data["id"] = test["id"]  # テストデータのIDを追加します。
# トークナイズした情報をdataフレームに追加します。
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)  # 各入力の長さを計算します。

# 拡張データフレームを初期化します。
aug_data = pd.DataFrame()
aug_data["id"] = test["id"]  # テストデータのIDを追加します。
# response_aとresponse_bを入れ替えたトークナイズを行います。
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)  # 各入力の長さを計算します。

In [ ]:
# トークンIDからデコードして、最初の入力を人間が読める形式に変換します。
print(tokenizer.decode(data["input_ids"][0]))

In [ ]:
# 拡張データフレームにおける最初の入力をトークンIDからデコードし、人間が読める形式に変換します。
print(tokenizer.decode(aug_data["input_ids"][0]))

# モデルの読み込み
各GPUに1つのモデルを読み込みます。

In [ ]:
# BitsAndBytesの設定を行います。
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットでモデルをロードします。
    bnb_8bit_compute_dtype=torch.float16,  # 計算時のデータ型をfloat16に設定します。
    bnb_8bit_use_double_quant=False,  # ダブル量子化を使用しない設定です。
)

# GPU 0にベースモデルをロードします。
device_0 = torch.device('cuda:0')  # 使用するデバイスを指定します。
base_model_0 = LlamaForSequenceClassification.from_pretrained(
    cfg.model_name,  # モデルの名前を指定します。
    num_labels=3,  # 出力ラベルの数を指定します。
    torch_dtype=torch.float16,  # トークンのデータ型をfloat16に設定します。
    quantization_config=bnb_config,  # 量子化構成を設定します。
    device_map='cuda:0'  # モデルをCUDAデバイス0にマッピングします。
)
base_model_0.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンのIDを設定します。

# GPU 1にベースモデルをロードします。
device_1 = torch.device('cuda:1')  # 使用するデバイスを指定します。
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    cfg.model_name,  # モデルの名前を指定します。
    num_labels=3,  # 出力ラベルの数を指定します。
    torch_dtype=torch.float16,  # トークンのデータ型をfloat16に設定します。
    quantization_config=bnb_config,  # 量子化構成を設定します。
    device_map='cuda:1'  # モデルをCUDAデバイス1にマッピングします。
)
base_model_1.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンのIDを設定します。

# 重みのロード

In [ ]:
# LoRAの設定を行います。
peft_config = LoraConfig(
    r=16,  # LoRAのランクを指定します。
    lora_alpha=32,  # LoRAのアルファ値を設定します。
    lora_dropout=0.10,  # LoRAに使用するドロップアウト率を設定します。
    bias='none',  # バイアスの設定を指定します。
    inference_mode=True,  # 推論モードを有効にします。
    task_type=TaskType.SEQ_CLS,  # タスクのタイプをシーケンス分類に設定します。
    target_modules=['o_proj', 'v_proj']  # 対象モジュールを指定します。
)

In [ ]:
# PEFTモデルを取得します。
model_0 = get_peft_model(base_model_0, peft_config).to(device_0)  # base_model_0にPEFT設定を適用し、デバイス0に移動します。
# 重みをロードします。
model_0.load_state_dict(torch.load(cfg.weights_path), strict=False)  # 重みを指定されたパスからロードします。
model_0.eval()  # モデルを評価モードに設定します。

# PEFTモデルを取得します。
model_1 = get_peft_model(base_model_1, peft_config).to(device_1)  # base_model_1にPEFT設定を適用し、デバイス1に移動します。
model_1.load_state_dict(torch.load(cfg.weights_path), strict=False)  # 重みを指定されたパスからロードします。
model_1.eval()  # モデルを評価モードに設定します。

In [ ]:
# モデルの学習可能なパラメータを表示します。
model_0.print_trainable_parameters()  # model_0の学習可能なパラメータを表示します。
model_1.print_trainable_parameters()  # model_1の学習可能なパラメータを表示します。

# 推論

In [ ]:
# 推論を行う関数を定義します。
@torch.no_grad()  # 勾配計算を無効にします。推論時は必要ないため効率的です。
@torch.cuda.amp.autocast()  # 自動混合精度を使用して計算を高速化します。
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝者の確率を格納するリストを初期化します。
    
    # データフレームをバッチサイズごとに処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終了インデックスを計算します。
        tmp = df.iloc[start_idx:end_idx]  # 現在のバッチのデータを取得します。
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換します。
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクをリストに変換します。
        
        # パディングを行い、モデルの入力形式に変換します。
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding=True,
            max_length=max_length,
            pad_to_multiple_of=None,
            return_tensors="pt",  # PyTorchテンソルとして返します。
        )
        
        # モデルを使って推論を行います。
        outputs = model(**inputs.to(device))  # デバイスに入力を移動させてモデルに渡します。
        proba = outputs.logits.softmax(-1).cpu()  # ロジットにソフトマックスを適用し、確率を計算します。

        # 各モデルの勝者の確率をリストに追加します。
        a_win.extend(proba[:, 0].tolist())  # モデルAの勝者の確率を追加します。
        b_win.extend(proba[:, 1].tolist())  # モデルBの勝者の確率を追加します。
        tie.extend(proba[:, 2].tolist())    # 引き分けの確率を追加します。
    
    # データフレームに勝者の情報を追加します。
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df  # 更新されたデータフレームを返します。

In [ ]:
# 時間計測を開始します。
st = time.time()

# 動的パディングを最大限活用するために入力の長さでソートします。
data = data.sort_values("length", ascending=False)
# サブセット1とサブセット2ではトークンの総数がほぼ同じである必要があります。
sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをサブセット1にコピーします。
sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをサブセット2にコピーします。

# スレッドプールエグゼキュータを使用して並行処理を行います。
with ThreadPoolExecutor(max_workers=2) as executor:
    # inference関数をサブセット1とサブセット2に対してモデル0とモデル1で実行します。
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

# 結果をデータフレームとして結合します。
result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 勝者の確率を抽出します。

# 経過時間を表示します。
print(f"経過時間: {time.time() - st}")

In [ ]:
# 時間計測を開始します。
st = time.time()

# テスト時間の拡張（TTA）が有効な場合の処理です。
if cfg.tta:
    # 入力の長さでソートし、処理速度を向上させます。
    data = aug_data.sort_values("length", ascending=False)
    sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをサブセット1にコピーします。
    sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをサブセット2にコピーします。

    # スレッドプールエグゼキュータを使用して並行処理を行います。
    with ThreadPoolExecutor(max_workers=2) as executor:
        # inference関数をサブセット1とサブセット2に対してモデル0とモデル1で実行します。
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    # 結果をデータフレームとして結合します。
    tta_result_df = pd.concat(list(results), axis=0)
    # TTAの順序が反転していることを考慮します。
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values  
    # 元の結果とTTA結果を平均化します。
    proba = (proba + tta_proba) / 2

# 経過時間を表示します。
print(f"経過時間: {time.time() - st}")

In [ ]:
# 結果データフレームに、勝者の確率を更新します。
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝者の確率を追加します。
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝者の確率を追加します。
result_df.loc[:, "winner_tie"] = proba[:, 2]      # 引き分けの確率を追加します。

# 提出用データフレームを作成します。
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  
# 提出用データフレームをCSVファイルとして保存します。
submission_df.to_csv('submission.csv', index=False)  
# 提出用データフレームを表示します。
display(submission_df)